In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt
import re

import folium
import rasterio
import rasterio.mask

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeClassifier

In [ ]:
incoming_weather_size_draught = pd.read_csv('Data/Input_ML/incoming_weather_size_draught.csv')
incoming_weather_size = pd.read_csv('Data/Input_ML/incoming_weather_size.csv')
incoming_weather = pd.read_csv('Data/Input_ML/incoming_weather.csv')

In [ ]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
incoming_weather_size_draught['Direction'] = label_encoder.fit_transform(incoming_weather_size_draught['Direction'])
incoming_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(incoming_weather_size_draught['Wind_speed'])
incoming_weather_size_draught['Type'] = label_encoder.fit_transform(incoming_weather_size_draught['Type'])
#incoming_weather_size_draught['to_haven'] = label_encoder.fit_transform(incoming_weather_size_draught['to_haven'])
incoming_weather_size_draught

In [ ]:
leaving_weather_size_draught = pd.read_csv('Data/Input_ML/leaving_weather_size_draught.csv')
leaving_weather_size = pd.read_csv('Data/Input_ML/leaving_weather_size.csv')
leaving_weather = pd.read_csv('Data/Input_ML/leaving_weather.csv')

In [ ]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
leaving_weather_size_draught['Direction'] = label_encoder.fit_transform(leaving_weather_size_draught['Direction'])
leaving_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(leaving_weather_size_draught['Wind_speed'])
leaving_weather_size_draught['Type'] = label_encoder.fit_transform(leaving_weather_size_draught['Type'])
#leaving_weather_size_draught['from_haven'] = label_encoder.fit_transform(leaving_weather_size_draught['from_haven'])

In [ ]:
leaving_top = leaving_weather_size_draught['from_haven'].value_counts()
incoming_top = incoming_weather_size_draught['to_haven'].value_counts()
print(leaving_top[0:2])
print(incoming_top[0:2])

In [ ]:
top_leaving = 'EURO'
top_incoming = 'EURO'

harbor_leaving = leaving_weather_size_draught[leaving_weather_size_draught['from_haven']== top_leaving]
harbor_incoming = incoming_weather_size_draught[incoming_weather_size_draught['to_haven']== top_incoming]


In [ ]:
columns_leaving = [
    #'from_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]

columns_incoming = [
    #'to_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]


In [ ]:
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(harbor_leaving[columns_leaving], harbor_leaving['area_ID'], test_size=0.3, random_state=42)
#X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(leaving_weather_size_draught[columns_leaving], leaving_weather_size_draught['area_ID'], test_size=0.3, random_state=42)
data = X_from_lat_test

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize the Decision Tree Classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Train the classifier using the training data
decision_tree_classifier.fit(X_from_lat_train, y_from_lat_train)

# Make predictions on the test data
predictions = decision_tree_classifier.predict(X_from_lat_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_from_lat_test, predictions)

print("Accuracy of the Decision Tree Classifier: {:.2f}%".format(accuracy * 100))

# Add predictions to the original DataFrame

data['Predicted_Area_ID'] = predictions
data



In [ ]:
merged_df = pd.merge(data, leaving_weather_size_draught, left_index=True, right_index=True, how='left')
merged_df

In [ ]:
# Create a new empty list to store prediction_center values
prediction_centers = []

# Iterate through the rows of merged_df
for index, row in merged_df.iterrows():
    # Get the Predicted_Area_ID value from the current row of merged_df
    predicted_area_id = row['Predicted_Area_ID']
    
    # Find the corresponding row in incoming_weather_size_draught where area_ID matches Predicted_Area_ID
    matching_row = incoming_weather_size_draught[incoming_weather_size_draught['area_ID'] == predicted_area_id]
    
    # Check if a matching row is found
    if not matching_row.empty:
        # Get the value from the 'center' column in the matching row
        prediction_center = matching_row['center'].values[0]
    else:
        # If no matching row is found, assign a default value (you can change this as per your requirement)
        prediction_center = None
    
    # Append the prediction_center value to the list
    prediction_centers.append(prediction_center)

# Add the prediction_centers list as a new column to merged_df
merged_df['prediction_center'] = prediction_centers

In [ ]:
merged_df

In [ ]:
# Function to convert string representation of coordinates to actual coordinates
def parse_coordinates(coord_str):
    if coord_str:
        # Extract numerical values from the string (format: 'POINT (longitude latitude)')
        coordinates = re.findall(r"[-+]?\d*\.\d+|\d+", coord_str)
        lat, lon = map(float, coordinates)
        return lat, lon
    else:
        return None

# Convert 'from_location' and 'center' columns to actual coordinates
merged_df['to_location_coords'] = merged_df['to_location'].apply(parse_coordinates)
merged_df['center_coords'] = merged_df['prediction_center'].apply(parse_coordinates)

# Function to calculate distance between two points using geodesic
def calculate_distance(row):
    if row['to_location_coords'] and row['center_coords']:
        return geodesic(row['to_location_coords'], row['center_coords']).meters
    else:
        return None

# Apply the calculate_distance function to the DataFrame to compute distances
merged_df['distance_to_predicted_center'] = merged_df.apply(calculate_distance, axis=1)
moyenne_distance = merged_df['distance_to_predicted_center'].mean()

# Afficher la moyenne
print("Average distance between predicted and real locations: ", moyenne_distance, "meters")



In [ ]:
import graphviz
# DOT data
dot_data = tree.export_graphviz(DecisionTreeClassifier, out_file=None, 
                                feature_names=iris.feature_names,  
                                class_names=iris.target_names,
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph


In [ ]:
graph.render("decision_tree_graphivz")
'decision_tree_graphivz.png'

In [ ]:
from dtreeviz.trees import dtreeviz # remember to load the package

viz = dtreeviz(DecisionTreeClassifier, harbor_leaving[columns_leaving], harbor_leaving['area_ID'],
                target_name="target",
                feature_names=iris.feature_names,
                class_names=list(iris.target_names))

viz

viz.save("decision_tree.svg")

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def calculate_haversine_distance(point1, point2):
    # Extract latitude and longitude from the points
    lat1, lon1 = map(float, point1.strip('POINT ()').split())
    point2 = point2.replace('[', '').replace(']', '').replace(',', '')
    lat2, lon2 = point2
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Radius of the Earth in kilometers (mean value)
    radius = 6371.0
    distance = radius * c

    return distance